In [3]:
import asyncio
from mpyc.runtime import mpc
from mpyc import asyncoro
import mpyc
import mpyc.gmpy
import numpy as np
import statsmodels.api as sm

In [4]:
import pandas as pd
iris = sm.datasets.get_rdataset("iris")
y = iris.data['Species']
X = iris.data.drop(columns=['Species'])
X = sm.add_constant(X)
X = X.to_numpy(dtype=np.float64)
y = pd.get_dummies(y, drop_first=True)["virginica"].to_numpy(dtype=np.float64)

In [5]:
import asyncio
from mpyc.runtime import mpc
from mpyc import asyncoro
import mpyc.gmpy

def factorial(x):
    res = 1
    for i in range(1,x+1):
        res = res*i
    return res

def exp(z):
    ftol = 1e-5
    exp_val = _exp(mpc.if_else(z < 0, -1, 1) * z, ftol)
    return mpc.if_else(z < 0, 1/exp_val, exp_val)

@asyncoro.mpc_coro
async def _exp(z, ftol):
    res = 1
    for i in range(1, 50):
        term = (z**i)/factorial(i)
        res += term
        if await mpc.output(term < ftol): break
    return res

In [6]:
def log(x, q=3):
    ftol = 1e-5
    q = 4
    r = 0
    for _ in range(20):
        s = (x > q)
        x /= mpc.if_else(s, q, 1.0)
        r += 1 * s
    return _log(x, ftol) + r * 1.3862943611198906188

@asyncoro.mpc_coro
async def _log(x, ftol):
    ratio = (x-1)/(x+1)
    res = 0
    for i in range(1,100,2):
        term = 2*(ratio**i)/i
        res += term
        if await mpc.output(mpc.abs(term) < ftol): break
    return res


In [7]:
class SecureLogistic:

    def __init__(self, y, X):
        self.secfloat = mpc.SecFxp(128)
        if type(y) is np.ndarray:
            y = self.secfloat.array(y)
        if type(X) is np.ndarray:
            X = self.secfloat.array(X)
        self.y = y
        self.X = X
        self.n_obs = len(y)

    def fit(self, maxiter=1):
        beta = self.secfloat.array(np.zeros(self.X.shape[1]))
        for i in range(maxiter):
            score = self.score(beta)
            hessian = self.hessian(beta)
            beta += self.np_inv(hessian) @ score
            print(f"iter {i} completed")
        return beta

    def likelihood(self):
        pass

    def score(self, beta):
        Xb = self.X @ beta
        exp_Xb = self.apply_exp(-Xb)
        self.S = 1/(1+exp_Xb)
        return (self.y - self.S) @ self.X / self.n_obs

    def hessian(self, beta):
        return mpc.np_multiply(self.X.T, self.S * (1 - self.S)) @ self.X / self.n_obs

    def apply_exp(self, X):
        X = X.copy()
        for i in range(len(X)):
            mpc.np_update(X, i, exp(X[i]))
        return X
    
    def np_inv(self, A, niter=100):
        n = A.shape[0]
        X_k = self.secfloat.array(np.eye(n)) / self.norm_1(A)
        for k in range(niter):
            X_k_next = 2 * X_k - X_k @ A @ X_k     
            X_k = X_k_next    
        return X_k_next
        
    def norm_1(self, A):
        n = mpc.np_sgn(A) * A
        n = mpc.np_sum(n, axis=0)
        n = mpc.np_amax(n)
        return n

In [8]:
model = SecureLogistic(y, X)

await mpc.output(model.fit(maxiter=15))

iter 0 completed
iter 1 completed
iter 2 completed
iter 3 completed
iter 4 completed
iter 5 completed
iter 6 completed
iter 7 completed
iter 8 completed
iter 9 completed
iter 10 completed
iter 11 completed
iter 12 completed
iter 13 completed
iter 14 completed


array([-42.6381451 ,  -2.46522212,  -6.68093344,   9.4294395 ,
        18.28626603])

In [9]:
sm.Logit(y, X).fit(maxiter=15).params

Optimization terminated successfully.
         Current function value: 0.039662
         Iterations 14


array([-42.63780381,  -2.4652202 ,  -6.68088701,   9.42938515,
        18.28613689])